##### Environment variable setup for Langchain, Langchain Smith and Groq

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

# To call groq open api models
os.environ["GROQ_APY_KEY"] = os.getenv("GROQ_APY_KEY")

# To use LangSmith
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

##### Sqlite DB connection and test

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

##### Use llama3 LLM model

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_APY_KEY")

#print(groq_api_key)
#Llama3-70b-8192
#Gemma2-9b-It
llm = ChatGroq(
    temperature=0,
    model="Llama3-70b-8192",
    groq_api_key=groq_api_key
)

##### Create Langchain SQL agent

In [4]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type=None, verbose=False)

##### Invoke natual language queries

In [5]:
agent_executor.invoke(
    {
        "input": "How many employees are there"
    }
)

{'input': 'How many employees are there', 'output': 'There are 8 employees.'}

In [35]:
agent_executor.invoke(
    {
        "input": "List the total sales per country."
    }
)

{'input': 'List the total sales per country.',
 'output': "The total sales per country are: [('Argentina', 37.62), ('Australia', 37.62), ('Austria', 42.62), ('Belgium', 37.62), ('Brazil', 190.1), ('Canada', 303.96), ('Chile', 46.62), ('Czech Republic', 90.24), ('Denmark', 37.62), ('Finland', 41.62), ('France', 195.1), ('Germany', 156.48), ('Hungary', 45.62), ('India', 75.26), ('Ireland', 45.62), ('Italy', 37.62), ('Netherlands', 40.62), ('Norway', 39.62), ('Poland', 37.62), ('Portugal', 77.24), ('Spain', 37.62), ('Sweden', 38.62), ('USA', 523.06), ('United Kingdom', 112.86)]"}

In [36]:
agent_executor.invoke(
    {
        "input": "Which country's customers spent the most?"
    }
)

{'input': "Which country's customers spent the most?",
 'output': 'The country whose customers spent the most is USA, with a total amount of 523.06.'}

In [37]:
agent_executor.invoke({"input": "Describe the playlisttrack table"})

{'input': 'Describe the playlisttrack table',
 'output': 'The PlaylistTrack table has two columns: PlaylistId and TrackId, both of which are integers and not null. The primary key is a composite key of PlaylistId and TrackId. There are foreign key relationships with the Track and Playlist tables.'}

In [38]:
agent_executor.invoke({"input": "create RDF model for table customer"})

{'input': 'create RDF model for table customer',
 'output': 'Here is the RDF model for the "Customer" table:\n\n```\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix customer: <http://example.org/customer/> .\n\ncustomer:Customer a rdf:Class .\n\ncustomer:CustomerId a rdf:Property ;\n  rdfs:domain customer:Customer ;\n  rdfs:range xsd:integer .\n\ncustomer:FirstName a rdf:Property ;\n  rdfs:domain customer:Customer ;\n  rdfs:range xsd:string .\n\ncustomer:LastName a rdf:Property ;\n  rdfs:domain customer:Customer ;\n  rdfs:range xsd:string .\n\ncustomer:Company a rdf:Property ;\n  rdfs:domain customer:Customer ;\n  rdfs:range xsd:string .\n\ncustomer:Address a rdf:Property ;\n  rdfs:domain customer:Customer ;\n  rdfs:range xsd:string .\n\ncustomer:City a rdf:Property ;\n  rdfs:domain customer:Customer ;\n  rdfs:range xsd:string .\n\ncustomer:State a rdf:Property ;\n  rdfs:domain customer:Customer ;\n  rdfs:range xsd:string .\n\ncustomer:Country a rdf:Property ;\n  

In [45]:
agent_executor.invoke(
    {
        "input": "Write a SQL query to show the list of invoices where total is less than average total from invoice table"
    }
)

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01j37ktt6nf429rshr7kj308vt` on tokens per minute (TPM): Limit 6000, Used 0, Requested 6678. Please try again in 6.78s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [44]:
agent_executor.invoke(
    {
        "input": "Write a SQL query to show the top 10 albums with the highest number of tracks. The result should contain 2 columns: album ID and number of tracks in the album name the column Tracks."
    }
)

{'input': 'Write a SQL query to show the top 10 albums with the highest number of tracks. The result should contain 2 columns: album ID and number of tracks in the album name the column Tracks.',
 'output': 'The SQL query to show the top 10 albums with the highest number of tracks is: SELECT AlbumId, COUNT(TrackId) AS Tracks FROM Track GROUP BY AlbumId ORDER BY Tracks DESC LIMIT 10;'}